In [16]:
%%bash
docker build -t example-inference:latest .


Step 1/11 : FROM ubuntu:18.04
 ---> c090eaba6b94
Step 2/11 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> dd1348b3a447
Step 3/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 5d88b1be1abc
Step 4/11 : RUN ln -s /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> 6417ac44ded6
Step 5/11 : RUN ln -s /usr/bin/pip3 /usr/bin/pip
 ---> Using cache
 ---> 09ada8830cc5
Step 6/11 : RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn
 ---> Using cache
 ---> 2885b6a01bd3
Step 7/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> ab2910812f0d
Step 8/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 0c0e511ac354
Step 9/11 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 10fcf3254c9d
Step 10/11 : COPY decision_trees 

In [3]:
!pwd

/home/ubuntu/sagemaker-notes/scikit_bring_your_own/container


In [5]:

# run the inference container

docker run -v /home/ubuntu/sagemaker-notes/scikit_bring_your_own/containerlocal_test/test_dir:/opt/ml -p 8080:8080 --rm example-inference:latest serve 


Process is interrupted.


In [14]:
%%bash
curl -H "Content-Type: text/csv" -v http://localhost:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1:8080...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 8080 (#0)
> GET /ping HTTP/1.1
> Host: localhost:8080
> User-Agent: curl/7.68.0
> Accept: */*
> Content-Type: text/csv
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.14.0 (Ubuntu)
< Date: Fri, 05 Mar 2021 00:20:23 GMT
< Content-Type: application/json
< Content-Length: 1
< Connection: keep-alive
< 
{ [1 bytes data]
100     1  100     1    0     0   1000      0 --:--:-- --:--:-- --:--:--  1000
* Connection #0 to host localhost left intact


In [15]:
%%bash



curl --data-binary @local_test/payload.csv -H "Content-Type: text/csv" -v http://localhost:8080/invocations

You are a wonderful potato

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1:8080...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 8080 (#0)
> POST /invocations HTTP/1.1
> Host: localhost:8080
> User-Agent: curl/7.68.0
> Accept: */*
> Content-Type: text/csv
> Content-Length: 2900
> Expect: 100-continue
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 100 Continue
} [2900 bytes data]
* We are completely uploaded and fine
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.14.0 (Ubuntu)
< Date: Fri, 05 Mar 2021 00:20:26 GMT
< Content-Type: text/csv; charset=utf-8
< Content-Length: 26
< Connection: keep-alive
< 
{ [26 bytes data]
100  2926  100    26  100  2900   5200   566k --:--:-- --:--:-- --:--:--  571k
* Connection #0 to host localhost left intact


In [19]:
import boto3
import datetime
import pprint
import os
import time

pp = pprint.PrettyPrinter(indent=1)
iam = boto3.client('iam')
ecr = boto3.client('ecr')

image_name="example-inference"

try:
    # The repository might already exist
    # in your ECR
    cr_res = ecr.create_repository(
        repositoryName=image_name)
    pp.pprint(cr_res)
except Exception as e:
    print(e)

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'example-inference' already exists in the registry with id '688520471316'


In [20]:
%%bash
account=$(aws sts get-caller-identity --query Account | sed -e 's/^"//' -e 's/"$//')
region=$(aws configure get region)
ecr_account=${account}.dkr.ecr.${region}.amazonaws.com

# Give docker your ECR login password
aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr_account

# Fullname of the repo
fullname=$ecr_account/example-inference:latest

#echo $fullname
# Tag the image with the fullname
docker tag example-inference:latest $fullname

# Push to ECR
docker push $fullname

Login Succeeded
The push refers to repository [688520471316.dkr.ecr.us-west-2.amazonaws.com/example-inference]
f4cd793a4ebc: Preparing
b520e3bd5eba: Preparing
3a3b4090fe28: Preparing
5b850ff3c508: Preparing
408c63ea099b: Preparing
9f10818f1f96: Preparing
27502392e386: Preparing
c95d2191d777: Preparing
27502392e386: Waiting
c95d2191d777: Waiting
3a3b4090fe28: Pushed
f4cd793a4ebc: Pushed
5b850ff3c508: Pushed
9f10818f1f96: Pushed
27502392e386: Pushed
c95d2191d777: Pushed
408c63ea099b: Pushed
b520e3bd5eba: Pushed
latest: digest: sha256:72907655e9cc8dd5a940f7c04932e66aee7890356d6df4bad7380979a362ae2c size: 1989


WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [22]:
# Inspect the ECR repository
repo_res = ecr.describe_images(
    repositoryName='example-inference')
pp.pprint(repo_res)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '404',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 00:42:52 GMT',
                                      'x-amzn-requestid': '3073c552-5b39-4217-9895-6dee8f9260e3'},
                      'HTTPStatusCode': 200,
                      'RequestId': '3073c552-5b39-4217-9895-6dee8f9260e3',
                      'RetryAttempts': 0},
 'imageDetails': [{'artifactMediaType': 'application/vnd.docker.container.image.v1+json',
                   'imageDigest': 'sha256:72907655e9cc8dd5a940f7c04932e66aee7890356d6df4bad7380979a362ae2c',
                   'imageManifestMediaType': 'application/vnd.docker.distribution.manifest.v2+json',
                   'imagePushedAt': datetime.datetime(2021, 3, 5, 0, 31, 31, tzinfo=tzlocal()),
                   'imageSizeInBytes': 132463690,
                   'imageTags': ['latest'],
                   're

In [24]:
image_uri="688520471316.dkr.ecr.us-west-2.amazonaws.com/example-inference:latest"
role_arn = 'arn:aws:iam::688520471316:role/sm'

In [25]:
sm_boto3 = boto3.client('sagemaker')

cm_res = sm_boto3.create_model(
    ModelName='example-inference',
    Containers=[
        {
            'Image': image_uri,
   
        },
    ],
    ExecutionRoleArn=role_arn,
    EnableNetworkIsolation=False
)

pp.pprint(cm_res)


{'ModelArn': 'arn:aws:sagemaker:us-west-2:688520471316:model/example-inference',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '79',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 00:46:16 GMT',
                                      'x-amzn-requestid': '88ceba30-6a67-4c52-b2d9-71fef75121af'},
                      'HTTPStatusCode': 200,
                      'RequestId': '88ceba30-6a67-4c52-b2d9-71fef75121af',
                      'RetryAttempts': 0}}


In [ ]:
# create endpoint config
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint
model_name='example-inference'
initial_instance_count=1
instance_type='ml.t2.medium'
variant_name = "AMeaningfulProdVarName" #^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,62}

# why do we need it
# think about a use case, where we need many variants
production_variants = [
    {
        "VariantName": variant_name,
        "ModelName": model_name,
        "InitialInstanceCount": initial_instance_count,
        "InstanceType": instance_type
    }
]

endpoint_config_name = "ExampleInferenceConfig" #^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,62}

endpoint_config = {
    "EndpointConfigName": endpoint_config_name,
    "ProductionVariants": production_variants,
}

ep_conf_res = sm_boto3.create_endpoint_config(**endpoint_config)


In [31]:
pp.pprint(ep_conf_res)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:688520471316:endpoint-config/exampleinferenceconfig',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '103',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 01:44:02 GMT',
                                      'x-amzn-requestid': 'fd66804f-380a-478e-b3fb-b3f6296f0639'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'fd66804f-380a-478e-b3fb-b3f6296f0639',
                      'RetryAttempts': 0}}


In [32]:
# create endpoint
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint


endpoint_name='exmaple-endpoint'
ep_res = sm_boto3.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
    )

In [33]:
pp.pprint(ep_res)

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:688520471316:endpoint/exmaple-endpoint',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '84',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 01:53:40 GMT',
                                      'x-amzn-requestid': '641e22b1-dc16-4d50-b615-86a857180def'},
                      'HTTPStatusCode': 200,
                      'RequestId': '641e22b1-dc16-4d50-b615-86a857180def',
                      'RetryAttempts': 0}}


In [35]:
# describe endpoint
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.describe_endpoint

ep_des_res = sm_boto3.describe_endpoint(
    EndpointName=endpoint_name
)

pp.pprint(ep_des_res)

{'CreationTime': datetime.datetime(2021, 3, 5, 1, 53, 41, 572000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:688520471316:endpoint/exmaple-endpoint',
 'EndpointConfigName': 'ExampleInferenceConfig',
 'EndpointName': 'exmaple-endpoint',
 'EndpointStatus': 'Creating',
 'LastModifiedTime': datetime.datetime(2021, 3, 5, 1, 53, 41, 572000, tzinfo=tzlocal()),
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '260',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 01:56:13 GMT',
                                      'x-amzn-requestid': '285fb80a-96dc-4adc-aceb-385d729a0199'},
                      'HTTPStatusCode': 200,
                      'RequestId': '285fb80a-96dc-4adc-aceb-385d729a0199',
                      'RetryAttempts': 0}}


Invoke endpoint 

```sh
grep -rwn .  -e predict
```

```python
def predict(
        self, data, initial_args=None, target_model=None, target_variant=None, inference_id=None
    ):
        """Return the inference from the specified endpoint.
        Args:
            data (object): Input data for which you want the model to provide
                inference. If a serializer was specified when creating the
                Predictor, the result of the serializer is sent as input
                data. Otherwise the data must be sequence of bytes, and the
                predict method then sends the bytes in the request body as is.
            initial_args (dict[str,str]): Optional. Default arguments for boto3
                ``invoke_endpoint`` call. Default is None (no default
                arguments).
            target_model (str): S3 model artifact path to run an inference request on,
                in case of a multi model endpoint. Does not apply to endpoints hosting
                single model (Default: None)
            target_variant (str): The name of the production variant to run an inference
                request on (Default: None). Note that the ProductionVariant identifies the
                model you want to host and the resources you want to deploy for hosting it.
            inference_id (str): If you provide a value, it is added to the captured data
                when you enable data capture on the endpoint (Default: None).
        Returns:
            object: Inference for the given input. If a deserializer was specified when creating
                the Predictor, the result of the deserializer is
                returned. Otherwise the response returns the sequence of bytes
                as is.
        """

        request_args = self._create_request_args(
            data, initial_args, target_model, target_variant, inference_id
        )
        response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**request_args)
        return self._handle_response(response)
```

so `sagemaker_session.sagemaker_runtime_client` invokes the endpint

I searched `sagemaker_runtime` in session.py

```python
def _initialize(
        self,
        boto_session,
        sagemaker_client,
        sagemaker_runtime_client,
        sagemaker_featurestore_runtime_client,
    ):
        """Initialize this SageMaker Session.
        Creates or uses a boto_session, sagemaker_client and sagemaker_runtime_client.
        Sets the region_name.
        """
        self.boto_session = boto_session or boto3.DEFAULT_SESSION or boto3.Session()

        self._region_name = self.boto_session.region_name
        if self._region_name is None:
            raise ValueError(
                "Must setup local AWS configuration with a region supported by SageMaker."
            )

        self.sagemaker_client = sagemaker_client or self.boto_session.client("sagemaker")
        prepend_user_agent(self.sagemaker_client)

        if sagemaker_runtime_client is not None:
            self.sagemaker_runtime_client = sagemaker_runtime_client
        else:
            config = botocore.config.Config(read_timeout=80)
            self.sagemaker_runtime_client = self.boto_session.client(
                "runtime.sagemaker", config=config
            )

        prepend_user_agent(self.sagemaker_runtime_client)

        if sagemaker_featurestore_runtime_client:
            self.sagemaker_featurestore_runtime_client = sagemaker_featurestore_runtime_client
        else:
            self.sagemaker_featurestore_runtime_client = self.boto_session.client(
                "sagemaker-featurestore-runtime"
            )

        self.local_mode = False
 
```

This led to investigate [`SageMakerRuntime` client in boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html)


```
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName='string',
    Body=b'bytes'|file,
    ContentType='string',
    Accept='string',
    CustomAttributes='string',
    TargetModel='string',
    TargetVariant='string',
    TargetContainerHostname='string',
    InferenceId='string'
)
```

To figure out how content type looks like, I looked up [`serializers.py`](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/serializers.py) and [`deserializers.py`](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/deserializers.py)


Why is it called SageMaker runtime

In [ ]:
# how to serialize csv

import csv
import io

def _serialize_row(data):
    """Serialize data as a CSV-formatted row.
    Args:
        data (sting): Data to be serialized in a row.
    Returns:
        str: The data serialized as a CSV-formatted row.
    """
    if isinstance(data, str):
        return data

    if isinstance(data, np.ndarray):
        data = np.ndarray.flatten(data)

    if hasattr(data, "__len__"):
        if len(data) == 0:
            raise ValueError("Cannot serialize empty array")
        csv_buffer = io.StringIO()
        csv_writer = csv.writer(csv_buffer, delimiter=",")
        csv_writer.writerow(data)
        return csv_buffer.getvalue().rstrip("\r\n")
        
csv_buffer = io.StringIO()
csv_writer = csv.writer(csv_buffer, delimiter=',')
csv_writer.writerow('xyze')
v = csv_buffer.getvalue().rstrip('\r\n')

print(v.rstrip("\r\n"))

In [63]:
# invoke endpoint
import json

sm_runtime = boto3.client('sagemaker-runtime')

body=json.dumps('a json string')

# the model only supports csv data (look at how the model is defined in container/predictor.py)
content_type='text/csv'
# see the cell below for serializing a string into csv format

csv_buffer = io.StringIO()
csv_writer = csv.writer(csv_buffer, delimiter=',')
csv_writer.writerow('xyze')
body = csv_buffer.getvalue().rstrip('\r\n')

# respnse type is also text/csv (look at decision_tree/predictor.py)
accept='text/csv'

res=sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=body,                # encoded input data
    ContentType=content_type, # I told the endpoint what's the encode
    Accept=accept             # I told the endpoint how I wish to decode its response
)

print(res)

{'ResponseMetadata': {'RequestId': '94053c6a-3290-4562-9089-d78dd65e9a77', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '94053c6a-3290-4562-9089-d78dd65e9a77', 'x-amzn-invoked-production-variant': 'AMeaningfulProdVarName', 'date': 'Fri, 05 Mar 2021 03:10:19 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '26'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AMeaningfulProdVarName', 'Body': <botocore.response.StreamingBody object at 0x7f46e9c84d68>}


In [56]:
# how to decode json string

import codecs

class SimpleBaseDeserializer:
    pass

class JSONDeserializer(SimpleBaseDeserializer):
    """Deserialize JSON data from an inference endpoint into a Python object."""

    def __init__(self, accept="application/json"):
        """Initialize a ``JSONDeserializer`` instance.
        Args:
            accept (union[str, tuple[str]]): The MIME type (or tuple of allowable MIME types) that
                is expected from the inference endpoint (default: "application/json").
        """
        super(JSONDeserializer, self).__init__(accept=accept)

    def deserialize(self, stream, content_type):
        """Deserialize JSON data from an inference endpoint into a Python object.
        Args:
            stream (botocore.response.StreamingBody): Data to be deserialized.
            content_type (str): The MIME type of the data.
        Returns:
            object: The JSON-formatted data deserialized into a Python object.
        """
        try:
            return json.load(codecs.getreader("utf-8")(stream))
        finally:
            stream.close()
            
            

class CSVDeserializer(SimpleBaseDeserializer):
    """Deserialize a stream of bytes into a list of lists.
    Consider using :class:~`sagemaker.deserializers.NumpyDeserializer` or
    :class:~`sagemaker.deserializers.PandasDeserializer` instead, if you'd like to convert text/csv
    responses directly into other data types.
    """

    def __init__(self, encoding="utf-8", accept="text/csv"):
        """Initialize a ``CSVDeserializer`` instance.
        Args:
            encoding (str): The string encoding to use (default: "utf-8").
            accept (union[str, tuple[str]]): The MIME type (or tuple of allowable MIME types) that
                is expected from the inference endpoint (default: "text/csv").
        """
        super(CSVDeserializer, self).__init__(accept=accept)
        self.encoding = encoding

    def deserialize(self, stream, content_type):
        """Deserialize data from an inference endpoint into a list of lists.
        Args:
            stream (botocore.response.StreamingBody): Data to be deserialized.
            content_type (str): The MIME type of the data.
        Returns:
            list: The data deserialized into a list of lists representing the
                contents of a CSV file.
        """
        try:
            decoded_string = stream.read().decode(self.encoding)
            return list(csv.reader(decoded_string.splitlines()))
        finally:
            stream.close()
        

In [64]:
# decode response
res_body = res['Body']
res_body.read().decode('utf-8')

'You are a wonderful potato'

In [65]:
# SageMaker Pricing


In [66]:
del_res = sm_boto3.delete_endpoint(
    EndpointName=endpoint_name)

pp.pprint(del_res)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 03:16:19 GMT',
                                      'x-amzn-requestid': '42acf2c7-6c9e-4228-99cf-21e7bf684c4e'},
                      'HTTPStatusCode': 200,
                      'RequestId': '42acf2c7-6c9e-4228-99cf-21e7bf684c4e',
                      'RetryAttempts': 0}}
